Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt

### Method Set Up

The project description provides the following hint:

<span style="color:red;">

*For numerics, you could use something simple such as forward Euler combined with finite difference, but be sure to validate your choice of step size and other parameters.*

</span>


From this suggestion, we will start exploring the numerical solution with forward Euler.

##### Forward Euler Recap (diffusion only)
$$

U_m^{n + 1} = U_m^{n} + D\Delta t \frac{U_{m+1}^{n} - 2U_m^{n} + U_{m-1}^{n}}{(\Delta x )^2} \\ 

C = \frac{D\Delta t}{(\Delta x )^2} \\ 


U_m^{n + 1} = U_m^{n} + C[{U_{m+1}^{n} - 2U_m^{n} + U_{m-1}^{n}}] \\ 

$$

where $C < 0.5$ for stability.


Rewrite as matrix formulations (from Lecture Notes)

![](docs/forward_euler_matrix_formulation.png)



<!-- $
  \frac{U_m^{n + 1} - U_m^n}{\Delta t} = \frac{U_{m - 1}^n - 2 U_m^n + U_{m + 1}^n}{(\Delta x)^2} \quad \textrm{ for } m \in \left\{ 1, \ldots, M - 1 \right\}, n \in \left\{ 0, \ldots, N - 1 \right\},
$ -->

### Problem Set Up

##### Intravenous Injection

$$

C_t = D C_{xx} - v C_x \\

x \in (-\infty, \infty) \\

D = 0.1 m^2/s \\
v = 0.2 m/s \\

% U_m^{n + 1} = U_m^{n} + D\Delta t \frac{U_{m+1}^{n} - 2U_m^{n} + U_{m-1}^{n}}{(\Delta x )^2} \\ 
$$

